# Fractal Image Pipeline

This tutorial demonstrates **computed tables** by building an image processing pipeline for Julia fractals.

You'll learn:
- **Manual tables**: Parameters you define (experimental configurations)
- **Lookup tables**: Fixed reference data (processing methods)
- **Computed tables**: Automatically generated results via `populate()`
- **Many-to-many pipelines**: Processing every combination of inputs × methods

In [ ]:
import datajoint as dj
import numpy as np
from matplotlib import pyplot as plt

schema = dj.Schema('tutorial_fractal')

## Julia Set Generator

Julia sets are fractals generated by iterating $f(z) = z^2 + c$ for each point in the complex plane. Points that don't escape to infinity form intricate patterns.

In [ ]:
def julia(c, size=256, center=(0.0, 0.0), zoom=1.0, iters=256):
    """Generate a Julia set image."""
    x, y = np.meshgrid(
        np.linspace(-1, 1, size) / zoom + center[0],
        np.linspace(-1, 1, size) / zoom + center[1]
    )
    z = x + 1j * y
    img = np.zeros(z.shape)
    mask = np.ones(z.shape, dtype=bool)
    for _ in range(iters):
        z[mask] = z[mask] ** 2 + c
        mask = np.abs(z) < 2
        img += mask
    return img

In [ ]:
# Example fractal
plt.imshow(julia(-0.4 + 0.6j), cmap='magma')
plt.axis('off');

## Pipeline Architecture

```
JuliaSpec (Manual)     DenoiseMethod (Lookup)
     │                        │
     ▼                        │
JuliaImage (Computed)         │
     │                        │
     └────────┬───────────────┘
              ▼
      Denoised (Computed)
```

- **JuliaSpec**: Manual — parameters we define
- **JuliaImage**: Computed — generated from specs
- **DenoiseMethod**: Lookup — fixed set of algorithms
- **Denoised**: Computed — each image × each method

In [ ]:
@schema
class JuliaSpec(dj.Manual):
    """Parameters for generating Julia fractals."""
    definition = """
    spec_id : int8 unsigned
    ---
    c_real : float64          # Real part of c
    c_imag : float64          # Imaginary part of c  
    noise_level = 50 : float64
    """

In [ ]:
@schema
class JuliaImage(dj.Computed):
    """Generated fractal images with noise."""
    definition = """
    -> JuliaSpec
    ---
    image : <blob>            # Generated fractal image
    """
    
    def make(self, key):
        spec = (JuliaSpec & key).fetch1()
        img = julia(spec['c_real'] + 1j * spec['c_imag'])
        img += np.random.randn(*img.shape) * spec['noise_level']
        self.insert1({**key, 'image': img.astype(np.float32)})

In [ ]:
from skimage import filters, restoration
from skimage.morphology import disk

@schema
class DenoiseMethod(dj.Lookup):
    """Image denoising algorithms."""
    definition = """
    method_id : int8 unsigned
    ---
    method_name : varchar(20)
    params : <blob>
    """
    contents = [
        [0, 'gaussian', {'sigma': 1.8}],
        [1, 'median', {'radius': 3}],
        [2, 'tv', {'weight': 20.0}],
    ]

In [ ]:
@schema
class Denoised(dj.Computed):
    """Denoised images: each image × each method."""
    definition = """
    -> JuliaImage
    -> DenoiseMethod
    ---
    denoised : <blob>
    """
    
    def make(self, key):
        img = (JuliaImage & key).fetch1('image')
        method, params = (DenoiseMethod & key).fetch1('method_name', 'params')
        
        if method == 'gaussian':
            result = filters.gaussian(img, **params)
        elif method == 'median':
            result = filters.median(img, disk(params['radius']))
        elif method == 'tv':
            result = restoration.denoise_tv_chambolle(img, **params)
        else:
            raise ValueError(f"Unknown method: {method}")
            
        self.insert1({**key, 'denoised': result.astype(np.float32)})

In [ ]:
dj.Diagram(schema)

## Running the Pipeline

1. Insert specs into Manual table
2. Call `populate()` on Computed tables

In [ ]:
# Define fractal parameters
JuliaSpec.insert([
    {'spec_id': 0, 'c_real': -0.4, 'c_imag': 0.6},
    {'spec_id': 1, 'c_real': -0.74543, 'c_imag': 0.11301},
    {'spec_id': 2, 'c_real': -0.1, 'c_imag': 0.651},
    {'spec_id': 3, 'c_real': -0.835, 'c_imag': -0.2321},
])
JuliaSpec()

In [ ]:
# Generate all fractal images
JuliaImage.populate(display_progress=True)

In [ ]:
# View generated images
fig, axes = plt.subplots(1, 4, figsize=(12, 3))
for ax, row in zip(axes, JuliaImage()):
    ax.imshow(row['image'], cmap='magma')
    ax.set_title(f"spec_id={row['spec_id']}")
    ax.axis('off')
plt.tight_layout()

In [ ]:
# Apply all denoising methods to all images
Denoised.populate(display_progress=True)

In [ ]:
# 4 images × 3 methods = 12 results
print(f"JuliaImage: {len(JuliaImage())} rows")
print(f"DenoiseMethod: {len(DenoiseMethod())} rows")
print(f"Denoised: {len(Denoised())} rows")

In [ ]:
# Compare denoising methods on one image
spec_id = 0
original = (JuliaImage & {'spec_id': spec_id}).fetch1('image')

fig, axes = plt.subplots(1, 4, figsize=(14, 3.5))
axes[0].imshow(original, cmap='magma')
axes[0].set_title('Original (noisy)')

for ax, method_id in zip(axes[1:], [0, 1, 2]):
    result = (Denoised & {'spec_id': spec_id, 'method_id': method_id}).fetch1('denoised')
    method_name = (DenoiseMethod & {'method_id': method_id}).fetch1('method_name')
    ax.imshow(result, cmap='magma')
    ax.set_title(method_name)

for ax in axes:
    ax.axis('off')
plt.tight_layout()

## Key Points

| Table Type | Populated By | Use For |
|------------|-------------|--------|
| **Manual** | `insert()` | Experimental parameters, user inputs |
| **Lookup** | `contents` attribute | Fixed reference data, method catalogs |
| **Computed** | `populate()` | Derived results, processed outputs |

The pipeline automatically:
- Tracks dependencies (can't process an image that doesn't exist)
- Skips already-computed results (idempotent)
- Computes all combinations when multiple tables converge

In [ ]:
# Add a new spec — populate only computes what's missing
JuliaSpec.insert1({'spec_id': 4, 'c_real': -0.7, 'c_imag': 0.27})

JuliaImage.populate(display_progress=True)  # Only spec_id=4
Denoised.populate(display_progress=True)    # Only spec_id=4 × 3 methods

In [ ]:
# Cleanup
schema.drop(prompt=False)